**最优化理论**

很多机器学习算法最后都归结为求解最优化问题。最优化问题一般都是求解极大值或者极小值，对于连续可微函数，在数学上有明确的[求解方法](https://blog.csdn.net/lanchunhui/article/details/51371444)。**实际问题中，函数本身可能过于复杂，比如包含指数函数、对数函数以及根号等，使得求解偏导数等于0的类似$\mathbf{A}\mathbf{x}=\mathbf{b}$的方程组不是那么容易(可能无解或者计算成本过高)，所以我们只能通过迭代的方法求近似解，称为数值计算。**

迭代法从一个初始点 $X_{0}$ 开始，通过某个映射 h 从 $X_{k}$ 移动到下一个点 $X_{k+1}$ ，构造数列 $\{X_k\}_{k\in (1,n)}$，直到收敛到梯度为0的点处。即有下面的极限成立：

$$ \lim_{k \rightarrow + \infty} \nabla f(x_k) = 0$$


一般会利用一阶导数信息即梯度或者二阶导数信息即Hessian矩阵。迭代法的核心是得到迭代公式：

$$x_{k+1} = h(x_k)$$


# 最小二乘法与投影矩阵

## 解的个数

在线性回归中，问题会归结于求解

$$\mathbf{A}\mathbf{x}=\mathbf{b}$$

其中 $\mathbf{A}\in R^{m\times n},b\in R^m$。 

我们知道：

* 如果系数矩阵的秩小于增广矩阵的秩，即 $rank (\mathbf{A}) < rank (\mathbf{A},\mathbf{b})$ 时，矩阵无解；


* 如果系数矩阵的秩等于增广矩阵的秩并且列满秩，即 $rank (\mathbf{A}) = rank (\mathbf{A},\mathbf{b})=n$ 时，矩阵有唯一解；


* 如果系数矩阵的秩等于增广矩阵的秩并且非列满秩，即 $rank (\mathbf{A}) = rank (\mathbf{A},\mathbf{b})< n$,矩阵有无穷多个解。

## 最小二乘法

下面我们来考虑下

$$\mathbf{A}\mathbf{x}=\mathbf{b}$$

其中 $\mathbf{A}\in R^{m\times n},b\in R^m, m \geq n,rank (\mathbf{A})=n,b\notin R(\mathbf{A})$ 的情形。 

因为 $b\notin R(\mathbf{A})$,所以求解该方程是无解的。求解该方程就变成下面的问题：

$$\min_{\mathbf{x}}\Vert\mathbf{A}\mathbf{x}-\mathbf{b}\Vert^2$$

是一个线性最小二乘问题。

如果向量 $\mathbf{x}^\ast$ 使得 $\Vert\mathbf{A}\mathbf{x}-\mathbf{b}\Vert^2$ 最小，即对所有的 $\mathbf{x}\in R^n$，有

$$\Vert\mathbf{A}\mathbf{x}-\mathbf{b}\Vert^2 \geq \Vert\mathbf{A}\mathbf{x}^\ast-\mathbf{b}\Vert^2$$

则称 $\mathbf{x}^\ast$ 是 $\mathbf{A}\mathbf{x}=\mathbf{b}$ 的最小二乘解。

***
下面的结论来自Edwin K.P.Chong 等著的《最优化导论》的第3章(P21)：

* 矩阵 $\mathbf{A}\in R^{m\times n}$ 的值域空间或像空间记为

$$ R(\mathbf{A}) \triangleq \{\mathbf{A}\mathbf{x}:\mathbf{x}\in R^n\}$$;

* 矩阵 $\mathbf{A}\in R^{m\times n}$ 的零空间或核记为

$$N(\mathbf{A}) \triangleq \{\mathbf{x}\in R^n:\mathbf{A}\mathbf{x}=\mathbf{0}\}$$;


* **结论：** 对于任意矩阵 $\mathbf{A}$,总有 $R(\mathbf{A})^{\bot}=N(\mathbf{A}^{\bot})$ 和 $N(\mathbf{A})^{\bot}=R(\mathbf{A}^{\bot})$。


***

下面的三个定理来自 Edwin K.P.Chong 等著的《最优化导论》的第12章(P151)：

* **结论1**： $\mathbf{A}\in R^{m\times n}, m \geq n,rank (\mathbf{A})=n$，则有 $rank(A)=n\Leftrightarrow rank (\mathbf{A}^T\mathbf{A})=n$。


* **结论2：** 能够最小化 $\Vert\mathbf{A}\mathbf{x}-\mathbf{b}\Vert^2$ 的向量 $\mathbf{x}^\ast$ 具有唯一性，可通过求解方程组 $\mathbf{A}^T\mathbf{A}\mathbf{x}=\mathbf{A}^T\mathbf{b}$得到，即 

$$\mathbf{x}^\ast = (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\mathbf{b}$$


* **结论3：** 如果向量 $\mathbf{h}\in R(\mathbf{A})$ 使得 $\mathbf{h}-\mathbf{b}$ 正交于子空间 $R(\mathbf{A})$，那么 $$\mathbf{h}=\mathbf{A}\mathbf{x}^\ast=\mathbf{P}\mathbf{b}$$

对应的 $\mathbf{x}^\ast$ 为：

$$\mathbf{x}^\ast = (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\mathbf{b}$$

其中 $\mathbf{P} = \mathbf{A}(\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T$ 是向量 $\mathbf{b}$ 到值空间 $R(\mathbf{A})$ 的正交投影算子。

**对结论3的证明：**

对于向量 $\mathbf{h}\in R(\mathbf{A})$，存在 $\mathbf{x}^\ast\in R^m$ 使得 $\mathbf{h}=\mathbf{A}\mathbf{x}^\ast$。

由 $\mathbf{h}-\mathbf{b}$ 正交于子空间 $R(\mathbf{A})$ 知

$$\mathbf{h}-\mathbf{b} \in R(\mathbf{A})^{\bot}=N(\mathbf{A}^T)$$

所以有：

$$\mathbf{A}^T(\mathbf{h}-\mathbf{b})=\mathbf{A}^T(\mathbf{A}\mathbf{x}^\ast-\mathbf{b})=\mathbf{0}$$

即

$$\mathbf{A}^T\mathbf{A}\mathbf{x}^\ast = \mathbf{A}^T\mathbf{b}$$

由 $rank(\mathbf{A})=n$ 可知 $rank(\mathbf{A}^T\mathbf{A})=n$，也就是说 $\mathbf{A}^T\mathbf{A}$ 是可逆的，所以：

$$\mathbf{x}^\ast = (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\mathbf{b}$$

$$\mathbf{h} = \mathbf{A}\mathbf{x}^\ast = \mathbf{A}(\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\mathbf{b}=\mathbf{P}\mathbf{b}$$

## 几何观点

<img src='figure/touying0.jpg' width='500' height='400' align='left'>

秩为 n 的矩阵 $\mathbf{A}\in R^{m\times n}$ 所有列张成的子空间为 $\mathbf{A}$ 的值域空间 $R(\mathbf{A})$，是 $R^{m}$ 的一个 n 维子空间。

当 $\mathbf{b}\in R(\mathbf{A})$ 时方程组才有精确解：

* 当 $m=n$ 时，$\mathbf{b}\in R(\mathbf{A})$ 总是成立的；

* 当 $m > n$ 时，因为 子空间 $R(\mathbf{A})$ 非常“薄”(“薄”的含义是：对于任意一个向量 $\mathbf{x}\in R^{m}$，其属于子空间 $R(\mathbf{A})$ 的概率非常低)，所以 $\mathbf{b}\in R(\mathbf{A})$ 的可能性比较小。

综上可知，在 $m \geq n$ 时从概率上讲方程 $\mathbf{A}\mathbf{x}=\mathbf{b}$ 有精确解的概率是比较低的。

假设 $\mathbf{b}$ 在 $R(\mathbf{A})$ 之外，那么寻找最小二乘解 $\Vert\mathbf{A}\mathbf{x}-\mathbf{b}\Vert^2 \geq \Vert\mathbf{A}\mathbf{x}^\ast-\mathbf{b}\Vert^2$ 本质上是在 $R(\mathbf{A})$ 上寻找向量 $\mathbf{h}$ 使得 $\Vert\mathbf{b} - \mathbf{h}\Vert$ 最小。


从几何意义上看，向量 $\mathbf{h}$ 应该使得 向量 $\mathbf{e}=\mathbf{b} - \mathbf{h}$ 正交于子空间 $R(\mathbf{A})$。换言之，此时向量 $\mathbf{h}$ 为向量 $\mathbf{b}$ 在 $R(\mathbf{A})$ 上的正交投影向量。

## 另一种获取最小二乘解的方法

构造目标函数

$$\begin{split}
f(\mathbf{x}) &=\Vert\mathbf{A}\mathbf{x}-\mathbf{b}\Vert^2\\ 
&=(\mathbf{A}\mathbf{x}-\mathbf{b})^T(\mathbf{A}\mathbf{x}-\mathbf{b})\\
&=\frac{1}{2}\mathbf{x}^T(2\mathbf{A}^T\mathbf{A})\mathbf{x} - \mathbf{x}^T(2\mathbf{A}^T\mathbf{b})+\mathbf{b}^T\mathbf{b}
\end{split}$$

对于二次函数 f,由于 $rank(\mathbf{A})=n$，所以二次项是正定的。利用局部极小值的一阶必要条件，可求得 f 的唯一极小点，满足

$$\nabla f(\mathbf{x}) = 2\mathbf{A}^T\mathbf{A}\mathbf{x} - 2\mathbf{A}^T\mathbf{b} = \mathbf{0}$$

该方程的唯一解为

$$\mathbf{x}^\ast = (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\mathbf{b}$$

# 梯度下降法

## 梯度下降法原理

**梯度下降法是通过沿着梯度下降（n=1对应的就是导数）的方向来求解近似极小值。**

我们下面针对一元**连续可微凸函数（也是应用梯度下降法的前提，分段连续可微是否可行呢？理论上也是可行的，分段求解即可）**（即必然有最小值）来推导梯度下降法的一般公式（一种解释[链接](http://www.cnblogs.com/ljygoodgoodstudydaydayup/p/7274943.html)，多元情形请参考[链接](https://zhuanlan.zhihu.com/p/36902908)）。一元函数的一阶泰勒公式展开为：

$$f(x + \Delta x) = f(x) + f^\prime(x)\Delta x + o(\Delta x)$$

即 
$$f(x + \Delta x) - f(x) = f^\prime(x)\Delta x + o(\Delta x)$$

为了保证每次迭代使得 $f(x + \Delta x) - f(x) < 0$ ,要有 $ f^\prime(x)\Delta x < 0 $ 。因为凸函数的梯度 $f^\prime(x) > 0 $，所以要保证 $\Delta x < 0$。

那么每次 $\Delta f = f(x + \Delta x) - f(x)$ 下降多少取决于 $|f^\prime(x)\Delta x|$ 的大小。对于一个固定的点 $ f^\prime(x) $ 是常数，所以 $\Delta f$ 的大小由 $\Delta x $ 决定。不妨设

$$ \Delta x = - \alpha f^\prime(x),\quad \alpha \in (0, 1)$$

则函数下降有：

$$ f(x + \Delta x) - f(x) = - \alpha f^\prime(x)^2$$

下降速度取决于每点的梯度大小以及学习率。

$ \Delta x = - \alpha f^\prime(x)$ 假设的合理性在于：
1. 保证了 $\Delta x $ 和 $ f^\prime(x) $方向相反，**沿着梯度相反方向下降是最快的方向**（多元情形下，两者都是向量，夹角是$\cos \theta$）；
2. $\alpha$ 作为学习率控制了下降的幅度，取值在0到1之间，是因为如果步长太大，会越过极小值。

下面是我自己画的草图，注意到 $f^\prime(x) = \tan \theta$ ， 考虑到 

$$ \tan \theta \approx \theta \quad \theta \rightarrow 0$$

当靠近极小值的时候，有$ \Delta x = - \alpha \theta$， 最终会收敛到极小值。

最终，求解极小值点的问题变成了按照图中斜率导数所示的“梯子”方向逐步下降直到最低点的问题。

<img src="figure/梯度下降.png" width="400" hegiht="300" align=left />

实际求解中会遇到局部极小值和鞍点问题，相应的解决办法我们稍后再谈。

## 梯度下降法

对于函数 $f: R^n\rightarrow R$，我们要最小化这个函数。

在 $\mathbf{u}$（单位向量）方向的方向导数是函数在 f 在 $\mathbf{u}$ 方向的斜率，即 $f(\mathbf{x}+\alpha\mathbf{u})$ 关于 $\alpha$ 在 $\alpha=0$ 时的导数，由链式法则可知 $\alpha=0$ 时，

$$\frac{\partial}{\partial\alpha}f(\mathbf{x}+\alpha\mathbf{u}) = \mathbf{u}^T\nabla_{\mathbf{x}}f(\mathbf{x})$$

为了最小化 f，我们希望找到使得 f 下降最快的方向。计算方向导数：

$$\begin{split}
&\min_{\mathbf{u}^T\mathbf{u}=1}\mathbf{u}^T\nabla_{\mathbf{x}}f(\mathbf{x})\\
=& \min_{\mathbf{u}^T\mathbf{u}=1}||\mathbf{u}||_2 ||\nabla_{\mathbf{x}} f(\mathbf{x})||_2\cos{\theta}
\end{split}$$

其中 $\theta$ 是 $\mathbf{u}$ 与梯度的夹角。将 $$||\mathbf{u}||_2=1$$ 带入，忽略与 $\mathbf{u}$ 无关的项，可以简化得到

$$\min_{\mathbf{u}}\cos{\theta}$$

在 $\mathbf{u}$ 与梯度方向相反时取得最小。

换句话说，梯度向量指向上坡，负梯度向量指向下坡。我们在负梯度方向移动可以减小 f。这就是 **梯度下降**。

梯度下降建议新的点为：

$$\mathbf{x}^\prime = \mathbf{x} - \epsilon \nabla_{\mathbf{x}}f(\mathbf{x})$$

其中 $\epsilon$ 为 **学习率**，是一个确定步长大小的正标量。

编程语言实现如下：
1. 确定学习率和初始的参数起点，学习率不能过大，否则会无法收敛；过小会导致计算太多，算法效率低；
2. 迭代：根据梯度下降参数迭代公式，算出新的参数向量，得到新的预测函数，进而得到新的成本；
3. 确认成本函数是否收敛：对比成本函数的变化，看成本是否越来越小，如果两次成本差值小于误差范围，停止迭代，否则继续。

## SGD 和 MBGD

***
### SGD


上面的梯度下降法中每次循环更新参数需要遍历所有的训练数据集，我们可以改成每次随机从训练数据集里抽取一个数据进行迭代计算，这就是**随机梯度下降法(Stochastic Gradient Descent，简称SGD)**，公式如下：

$$\theta_j = \theta_j - \alpha(h(\textbf{x}^{(i)})-y^{(i)})x_j^{(i)}$$

**随机梯度下降算法和梯度下降算法在数学上证明是等价的。**

SGD的优缺点：
* 对于训练数据size比较大的训练集，训练速度快，大概是 O(n) 和 $O(2^n)$ 的对比；
* 伴随的问题是噪音要多，使得SGD并不是每次迭代都向着整体最优化方向。但是由于更新多轮，整体方向还是大致朝着极小值方向更新。
* SGD和MBGD相比BGD,优点是提升了效率，但算法最终不会收敛到最优点，会在最优点附近波动。

为了更加接近最优值，可以逐渐降低学习率：

**开始时，走的每一步较大（这有助于快速前进同时跳过局部最小值），然后变得越来越小，从而使算法到达全局最小值**。 这个过程被称为**模拟退火。**

***
### MBGD

小批量梯度下降法（Mini-batch Gradient Descent，简称MBGD）是 BGD 和 SGD 的折中，为了解决批梯度下降法训练速度慢，以及随机梯度下降法的准确性综合而来,每次迭代选用m个小批量样本进行更新。

下面的算法展示了 沿着这个梯度的估计下降。
***
**MBGD** 小批量随机梯度下降在第 k 个训练迭代的更新
***
**Require:** 学习率 $\epsilon_k$ 和初始参数 $\mathbf{\theta}$

**While** 停止准则未满足 **do**
* 从训练集中采样包含 m 个样本的小批量集合 $\{\mathbf{x}_1,\mathbf{x}_2,\cdots,\mathbf{x}_m \}$,其中 $\mathbf{x}_i$ 对应的值为 $\mathbf{y}_i$;


* 计算梯度估计： $\hat{\mathbf{g}}\leftarrow +\frac{1}{m}\nabla_{\mathbf{\theta}}\sum_i L(f(\mathbf{x}_i;\mathbf{\theta}),\mathbf{y}_i)$;


* 应用更新：$\mathbf{\theta}\leftarrow \mathbf{\theta} - \epsilon \hat{\mathbf{g}}$

**end While**
***

SGD 及其变种算法的关键参数是**学习率**。之前我们介绍 SGD 使用固定的学习率，在实践中有必要随着时间点的推移逐渐降低学习率，因此我们记第 k 步迭代的学习率为 $\epsilon_k$。原因如下：

    当我们使用全批量梯度下降到达极小点时，整个代价函数的真实梯度会变得很小直到为0，所以全批量梯度下降可以使用固定的学习率。但是 MBGD 中梯度估计引入的噪声源(m 个训练样本的随即采样) 并不会在极小值点消失，所以学习率要逐渐降低。

H.Herbert 和 S.Monro 在论文 [A Stochastic Approximation Method](https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586) 中给出了保证 SGD及其变种收敛的充分条件是：

$$\begin{split}
& \sum_{k=1}^{\infty}\epsilon_k = \infty\\
& \sum_{k=1}^{\infty}\epsilon_k^2 = \infty
\end{split}$$

实践中，一般会选出**线性衰减学习率** 直到第 $\tau$ 次迭代：

$$\epsilon_k = (1-\alpha)\epsilon_0 + \alpha\epsilon_\tau = \epsilon_0 -\alpha(\epsilon_0 -\epsilon_\tau) $$

其中 $\alpha = \frac{k}{\tau}$。$\tau$ 步迭代后，一般使 $\epsilon$ 保持常数。

备注：线性衰减不满足SGD收敛的充分条件？？

**$\epsilon_0$、$\epsilon_\tau$、$\tau$ 的选择：**

* $\tau$ 通常被设为可以满足反复遍历训练集几百次的迭代次数，如果是SGD，则迭代次数是训练集规模与 $k*100$ 的乘积；


* $\epsilon_\tau$ 一般被设置为 $\epsilon_0*1\%$;


* 如果 $\epsilon_0$ 过大，学习曲线将剧烈震荡，代价函数会明显增加；相比剧烈震荡而言，温和的震荡是良好的，容易在训练随机代价函数时出现，例如使用 Dropout 的代价函数；如果学习率太小，学习过程会很缓慢，并且容易卡在一个相当高的代价值。

    如果初始学习率太低，学习会卡在一个相当高的代价值。就总训练时间和最终代价值而言，最优的初始学习率的效果会好于一般的学习率(??)大约迭代100次之后最佳的效果。所以通常最好是检测最早的几轮迭代，选择一个比在效果上表现最佳的学习率（？？）更大的学习率，但又不能太大导致严重的震荡，更像是一门艺术。

***
**衡量收敛率**


优化算法的收敛速率通过衡量**额外误差 $J(\mathbf{\theta})-\min_{\mathbf{\theta}}J(\mathbf{\theta})$**，即当前代价函数超出最低可能代价的数量。

SGD 应用于凸问题时，k 步迭代之后的额外误差量级是 $O(\frac{1}{\sqrt{k}})$（考虑 m 个均值的标准差为 $\frac{\sigma}{\sqrt{m}}$），强凸情形下是 $O(\frac{1}{k})$。如果想要进一步改进，需要假定额外的前提条件。

全批量梯度下降在理论上比随机梯度有更好的收敛率，但是 Cramer-Rao 界限指出，**泛化误差的下降速度不会快于 $O(\frac{1}{\sqrt{k}})$。** 另外，渐近分析没有考虑 SGD 在下降效率上的优点：**对于大数据集， SGD 只需要非常少量样本计算梯度从而实现初始快速更新，在时间上远远超过了其缓慢的渐近收敛。**

为了权衡全批量梯度下降和 SGD 两者的优点，我们可以在学习过程中逐渐增大 “小批量” 的大小。想要了解更多 SGD，可以参看 [Online Learning and Stochastic
Approximations](https://leon.bottou.org/publications/pdf/online-1998.pdf)

## 动量与 Nesterov 动量

### 动量

虽然 SGD 很受欢迎，但有时学习过程会很慢。**动量方法旨在加速学习，特别是处理高曲率、小但一致的梯度，或者是带噪声的梯度。动量算法积累了之前梯度指数级衰减的移动平均(??)，并且继续沿该方向移动。**如下图所示。

<img src='figure/momentum0.png'>

形式上看，动量算法引入了变量 $\mathbf{v}$ 充当速度角色——代表参数在参数空间移动的方向和速率。速度被设为负梯度的指数衰减平均。超参数 $\alpha\in [0,1)$ 决定了之前梯度的贡献衰减得有多快。更新规则如下： 

$$\begin{split}
& \mathbf{g}\leftarrow \epsilon\nabla_{\mathbf{\theta}}\left(\frac{1}{m}\sum_{i=1}^{m}L(f(\mathbf{x}_i;\mathbf{\theta}), \mathbf{y}_i)\right)\\
& \mathbf{v}_k\leftarrow a\mathbf{v}_{k-1} - \epsilon\mathbf{g}\\
& \mathbf{\theta}_k\leftarrow\mathbf{\theta}_{k-1}+\mathbf{v}_k
\end{split}$$

其中速度的更新等价于
$$\begin{split}
&\mathbf{v}_k - \frac{\epsilon\mathbf{g}}{\alpha-1} = \alpha(\mathbf{v}_{k-1} - \frac{\epsilon\mathbf{g}}{\alpha-1})\\
& \mathbf{v}_k = -\left(\frac{1-\alpha^k}{1-\alpha}\epsilon\mathbf{g} +\alpha^k\mathbf{v}_0\right)
\end{split}\tag{Mom-1}$$

由(Mom-1)的第二个式子可知，当 k 足够大时，因为$\alpha\in [0,1)$ 所以 

$$\mathbf{v}_k\rightarrow -\frac{1}{1-\alpha}\epsilon\mathbf{g}  \quad when \;\alpha^k\rightarrow 0$$

进而参数的更新为

$$\mathbf{\theta}_k\leftarrow\mathbf{\theta}_{k-1}-\frac{\epsilon\mathbf{g}}{1-\alpha}$$

相比之前的 SGD，其速度更新的步长只是学习率乘以梯度范数 $\mathbf{\theta}_k\leftarrow\mathbf{\theta}_{k-1}-\epsilon\mathbf{g}$,现在步长取决于梯度序列的大小和排列($\{\mathbf{g}_k\}$)。

上面的推导是基于连续的梯度指向相同的方向，此时步长最大。也就是说，如果动量算法总是观测到梯度 $\mathbf{g}$,则会在 $-\mathbf{g}$ 上不停加速，直到达到最终速度。其步长大小为

$$\frac{\epsilon||\mathbf{g}||}{1-\alpha}$$

**动量的超参数 $\alpha$ 选择**


因此，将动量的超参数视为 $\frac{1}{1-\alpha}$ 有助于理解。例如 $\alpha=0.9$ 最大对应着梯度下降算法的10倍速度。实践中 $\alpha$ 一般取值为 0.5、 0.9 和 0.99。和学习率一样，$\alpha$ 也会随着时间不断调整。一般初始值是一个较小的值，随后慢慢变大。**随着时间推移调整 $\alpha$ 没有收缩 $\epsilon$ 重要（？？）。**

注：实际迭代时每一步的梯度 $\mathbf{g}_k$ 是不同的，所以步长中系数并不是简单的 $\frac{\epsilon}{1-\alpha}$，这只是一种极端情况，也就是连续的梯度指向相同的方向。

动量算法的算法伪代码图如下：

***
**动量算法** 使用动量的随机梯度下降
***
**Require:** 学习率 $\epsilon$，动量参数 $\alpha$

**Require:** 初始参数 $\mathbf{\theta}$，初始速度 $\mathbf{v}$

**While** 停止准则未满足 **do**
* 从训练集中采样包含 m 个样本的小批量集合 $\{\mathbf{x}_1,\mathbf{x}_2,\cdots,\mathbf{x}_m \}$,其中 $\mathbf{x}_i$ 对应的值为 $\mathbf{y}_i$;


* 计算梯度估计： $\mathbf{g}\leftarrow \frac{1}{m}\nabla_{\mathbf{\theta}}\sum_i L(f(\mathbf{x}_i;\mathbf{\theta}),\mathbf{y}_i)$;


* 计算速度更新： $\mathbf{v}_k\leftarrow a\mathbf{v}_{k-1} - \epsilon\mathbf{g}$


* 应用更新：$\mathbf{\theta}_k\leftarrow \mathbf{\theta}_{k-1} + \mathbf{v}_k$

**end While**
***

### Nesterov 动量

受到 Nesterov 加速梯度算法的启发， Sutskever 提出了动量算法的一个变种，相应的更新规则如下：

$$\begin{split}
& \mathbf{g}\leftarrow \epsilon\nabla_{\mathbf{\theta}}\left(\frac{1}{m}\sum_{i=1}^{m}L(f(\mathbf{x}_i;\mathbf{\theta}+ a\mathbf{v}), \mathbf{y}_i)\right)\\
& \mathbf{v}_k\leftarrow a\mathbf{v}_{k-1} - \epsilon\mathbf{g}\\
& \mathbf{\theta}_k\leftarrow\mathbf{\theta}_{k-1}+\mathbf{v}_k
\end{split}$$

可以看出，两者的区别体现在梯度计算上。在 Nesterov 动量中，梯度计算在施加当前速度之后。因此，Nesterov 动量可以解释为往标准动量方法中添加了一个**校正因子**。完整的Nesterov 动量 如下所示。

***
**Nesterov 动量算法** 使用Nesterov 动量的随机梯度下降
***
**Require:** 学习率 $\epsilon$，动量参数 $\alpha$

**Require:** 初始参数 $\mathbf{\theta}$，初始速度 $\mathbf{v}$

**While** 停止准则未满足 **do**

* 从训练集中采样包含 m 个样本的小批量集合 $\{\mathbf{x}_1,\mathbf{x}_2,\cdots,\mathbf{x}_m \}$,其中 $\mathbf{x}_i$ 对应的值为 $\mathbf{y}_i$;


* 应用临时更新： $\mathbf{\hat{\theta}}\leftarrow \mathbf{\theta}+ \alpha\mathbf{v}$;


* 计算梯度估计： $\mathbf{g}\leftarrow \frac{1}{m}\nabla_{\mathbf{\hat{\theta}}}\sum_i L(f(\mathbf{x}_i;\mathbf{\hat{\theta}}),\mathbf{y}_i)$;


* 计算速度更新： $\mathbf{v}_k\leftarrow a\mathbf{v}_{k-1} - \epsilon\mathbf{g}$


* 应用更新：$\mathbf{\theta}_k\leftarrow \mathbf{\theta}_{k-1} + \mathbf{v}_k$

**end While**
***

在 凸批量梯度的情况下， Nesterov 动量将 额外误差收敛率从 $O(\frac{1}{k})$ （k 步后）改进到 $O(\frac{1}{k^2})$,但是在随机梯度的情况下， Nesterov 动量没有改进收敛率。

# 牛顿法

牛顿法是最广泛使用的二阶方法，它是基于二阶泰勒级数展开在某点$\mathbf{\theta}_0$ 附近来近似 $J(\mathbf{\theta})$ 的优化方法。忽略高阶导数：

$$J(\mathbf{\theta}) \approx J(\mathbf{\theta}_0) + (\mathbf{\theta}-\mathbf{\theta}_0)^T\nabla_{\mathbf{\theta}}J(\mathbf{\theta}_0)+\frac{1}{2}(\mathbf{\theta}-\mathbf{\theta}_0)^T\mathbf{H}(\mathbf{\theta}-\mathbf{\theta}_0)$$

其中 $\mathbf{H}$ 是 J 相对于 $\mathbf{\theta}$ 的 Hessian 矩阵在 $\mathbf{\theta}_0$ 处的估计。我们求解这个函数的临界点：

$$\frac{\partial J(\mathbf{\theta})}{\partial\mathbf{\theta}} = \mathbf{0}$$

得到

$$\mathbf{\theta}^\ast = \mathbf{\theta}_0-\mathbf{H}^{-1}\nabla_{\mathbf{\theta}}J(\mathbf{\theta}_0)$$

因此，对于局部的二次函数(具有正定的)$\mathbf{H}$，用 $\mathbf{H}^{-1}$ 重新调整梯度，牛顿法直接跳到极小值。

如果目标函数是凸的但是非二次的(有高阶项)，该更新是迭代的，得到和牛顿法相关的算法。算法如下所示。

<img src='figure/newton0.png'>

在深度学习中，目标函数通常非凸，如鞍点。这种情况我们可以通过正则化 Hessian 矩阵来避免，常用的正则化策略包括在 Hessian 矩阵对角线上增加常数 $\alpha$ 。加入正则化后解变为

$$\mathbf{\theta}^\ast = \mathbf{\theta}_0-[\mathbf{H}+\alpha\mathbf{I}]^{-1}\nabla_{\mathbf{\theta}}J(\mathbf{\theta}_0)$$

这个正则化策略用于牛顿法的近似，只要 Hessian 矩阵的负特征值比较接近零，效果还是会很好。

曲率方向极端的情况下， $\alpha$ 的值必须足够大，以抵消负特征值。然而，如果 $\alpha$ 持续增加， Hessian 矩阵会变成由 $\alpha\mathbf{I}$ 主导，通过牛顿法选择的方向会收敛到普通梯度的$\frac{1}{\alpha}$ 。 

***
**牛顿法的计算负担**

牛顿法用于训练大型神经网络还受限于明显的计算负担， Hessian 矩阵中元素数目是参数数量的平方。如果参数数目为 k，牛顿法要计算 $k\times k$ 矩阵的逆，计算复杂度为 $O(K^3)$。

另外，由于参数每次更新都会改变，每次训练迭代都要计算 Hessian 矩阵的逆。这就导致，只有参数很小的网络才能在实际中用牛顿法进行训练。

# 最速下降法

**等价的极值问题**

如果矩阵 $\mathbf{A}\in R^{n\times n}$ 是对称正定矩阵，则求解线性方程组

$$\mathbf{A}\mathbf{x}=\mathbf{b}$$

的解 $\mathbf{x}^\ast$ 等价于求解二次函数

$$\varphi(\mathbf{x})=(\mathbf{A}(\mathbf{x}-\mathbf{x}^\ast),(\mathbf{x}-\mathbf{x}^\ast))$$

的极小值点。

考虑 $R^n$ 中的二次函数

$$\eta=\varphi(\mathbf{x})=(\mathbf{A}(\mathbf{x}-\mathbf{x}^\ast),(\mathbf{x}-\mathbf{x}^\ast))$$

其中 $\mathbf{A}\in R^{n\times n}$ 是对称正定矩阵。

我们的目标是求上式的极小值。

这个二次函数几何上是椭圆抛物面，固定$\eta = eta_0, \; \varphi(\mathbf{x})=\eta_0$ 是 $R^n$ 中的超椭圆面， $\mathbf{x}^\ast$ 是此超椭圆面的中心。

给定初始近似 $\mathbf{x}^{(0)}$，在椭圆面 $\varphi(\mathbf{x})=\varphi(\mathbf{x^{(0)}})$ 上选择一个方向，使 $\varphi(\mathbf{x})$ 沿此方向下降最快。

设 $\mathbf{r}\in R^n$ 是任意给定的向量，考虑 $\varphi(\mathbf{x}^{(0)}+\alpha\mathbf{r})$，其中 $\alpha$ 是任意的实数。容易得到

$$\begin{split}
\frac{d}{d\alpha}\varphi(\mathbf{x}^{(0)}+\alpha\mathbf{r})|_{\alpha=0}
=& \frac{d}{d\alpha}[\alpha^2(\mathbf{A}\mathbf{r},\mathbf{r})+\alpha(\mathbf{A}\mathbf{r},\mathbf{x}_0-\mathbf{x}^\ast)+\alpha(\mathbf{r},\mathbf{A}(\mathbf{x}_0-\mathbf{x}^\ast))]|_{\alpha=0}\\
=& \frac{d}{d\alpha}[\alpha^2(\mathbf{A}\mathbf{r},\mathbf{r})+2\alpha(\mathbf{r},\mathbf{A}(\mathbf{x}_0-\mathbf{x}^\ast))]|_{\alpha=0}\\
=& [2\alpha(\mathbf{A}\mathbf{r},\mathbf{r})+2(\mathbf{r},\mathbf{A}(\mathbf{x}_0-\mathbf{x}^\ast))]|_{\alpha=0}\\
=& -2(\mathbf{A}(\mathbf{x}^\ast-\mathbf{x}_0),\mathbf{r})
\end{split}$$

其中，因为$\mathbf{A}=\mathbf{A}^T$， 所以

$$(\mathbf{A}\mathbf{r},\mathbf{x}_0-\mathbf{x}^\ast) = (\mathbf{A}\mathbf{r})^T(\mathbf{x}_0-\mathbf{x}^\ast) = \mathbf{r}^T\mathbf{A}(\mathbf{x}_0-\mathbf{x}^\ast) =(\mathbf{r},\mathbf{A}(\mathbf{x}_0-\mathbf{x}^\ast))$$

由上式可知，$\varphi(\mathbf{x}) $ 在 $\mathbf{x}^{(0)}$ 处的最速下降方向是 $\mathbf{r}^{(0)} = \mathbf{A}(\mathbf{x}^\ast-\mathbf{x}_0) = \mathbf{b}-\mathbf{A}\mathbf{x}^{(0)}$,称 $\mathbf{r}^{(0)}$ 是 $\mathbf{x}^{(0)}$ 的**残量**或者**残向量**。

然后在最速下降方向 $\mathbf{r}^{(0)}$ 上求一点，使得 $\varphi(\mathbf{x})$ 取极小值，即求 $\alpha_0$ 使得

$$\varphi(\mathbf{x}^{(0)}+\alpha_0\mathbf{r}^{(0)})=\min_{\alpha} \varphi(\mathbf{x}^{(0)}+\alpha\mathbf{r}^{(0)})$$

由

$$\frac{d}{d\alpha}\varphi(\mathbf{x}^{(0)}+\alpha\mathbf{r}^{(0)}) = -2(\mathbf{A}\mathbf{r}^{(0)},\mathbf{r}^{(0)})-\alpha(\mathbf{r}^{(0)},\mathbf{r}^{(0)})=0$$

解得

$$\alpha_0 = \frac{(\mathbf{r}^{(0)},\mathbf{r}^{(0)})}{(\mathbf{A}\mathbf{r}^{(0)},\mathbf{r}^{(0)})}$$

这表明 $\varphi(\mathbf{x}^{(0)}+\alpha\mathbf{r}^{(0)})$ 在

$$\mathbf{x}^{(1)} = \mathbf{x}^{(0)}+\alpha_0 \mathbf{r}^{(0)}$$

处达到最小。

对 $\mathbf{x}^{(1)}$ 重复上面的讨论，同样可以得到最速下降方向 $\mathbf{r}^{(1)}$ 以及在此方向上的极小值点 $\mathbf{x}^{(1)}$。

我们可以得到如下的递推关系：

$$\begin{split} 
&\mathbf{r}^{(k)} = \mathbf{b}-\mathbf{A}\mathbf{x}^{(k)}\\
&\alpha_k = \frac{(\mathbf{r}^{(k)},\mathbf{r}^{(k)})}{(\mathbf{A}\mathbf{r}^{(k)},\mathbf{r}^{(k)})}\\
&\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)}+\alpha_k\mathbf{r}^{(k)}
\end{split}$$

其中 $k=0,1,\cdots$

基于最速下降法的 $\{\mathbf{x}^{(k)}\},\{\mathbf{r}^{(k)}\}$ 有下列几何特征：

* $(\mathbf{r}^{(k+1)},\mathbf{r}^{(k)})=0$;


* $\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)}+\alpha_k\mathbf{r}^{(k)}$ 是 $\mathbf{r}^{(k)}$ 与 $\varphi(\mathbf{x})=\varphi(\mathbf{x}^{(k)})$ 相交所得线段的中点。

图示如下：

<img src='figure/steepestdescent0.png' width='400' height='300' align='left'>

# 共轭梯度法

在最速下降法中，$(\mathbf{r}^{(k+1)},\mathbf{r}^{(k)})=0$。所以 k+1 次的下降方向与 k 次的下降方向是正交的，不会选择保持前一个搜索方向的最小值。按照每次在给定点的最速下降方向下降，未必是最好的策略，因为这只是局部策略，为尽快求解，我们应考虑整体策略。也就是我们下面要介绍的共轭梯度法。

首先我们给出**共轭向量组**的定义：

设 $\mathbf{A}$ 对称正定，若 $R^n$ 中向量组

$$\{\mathbf{p}^{(0)},\mathbf{p}^{(1)},\cdots,\mathbf{p}^{(l)}\}\tag{CG-1}$$

满足

$$(\mathbf{A}\mathbf{p}^{(i)},\mathbf{p}^{(j)})=0,\; i\neq j\tag{CG-2}$$

则称其为 $R^n$ 中一个 **A 共轭向量组**，或 **A 正交向量组**。

显然，当 $l< n$ 时，不含零向量的 A 共轭向量组线性无关，当 $l=n-1$ 时，它们构成 $R^n$ 的一组 A 正交基。当$\mathbf{A}=\mathbf{I}$ 时， A 正交性就是一般的正交性。

将方程组 $\mathbf{A}\mathbf{x}=\mathbf{b}$ 的解 $\mathbf{x}^\ast$ 按 $\mathbf{x}^\ast$ 按 (CG-1)(取 $l=n-1$)展开：

$$\mathbf{x}^\ast=\sum_{i=0}^{n-1}\alpha_i\mathbf{p}^{(i)}$$,

带入方程组 $\mathbf{A}\mathbf{x}=\mathbf{b}$，得到

$$\sum_{i=0}^{n-1}\alpha_i\mathbf{A}\mathbf{p}^{(i)}=\mathbf{b}$$

两边与 $\mathbf{p}^{(i)}$ 作内积，由 A 正交性(CG-2) 得

$$\alpha_k = \frac{(\mathbf{b},\mathbf{p}^{(k)})}{(\mathbf{A}\mathbf{p}^{(k)},\mathbf{p}^{(k)})}, k=0,1,\cdots,n-1$$

所以，解向量

$$\mathbf{x}^\ast=\sum_{i=0}^{n-1}\frac{(\mathbf{b},\mathbf{p}^{(i)})}{(\mathbf{A}\mathbf{p}^{(i)},\mathbf{p}^{(i)})}\mathbf{p}^{(i)}$$

可以写成有限步迭代的形式

$$\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \alpha_k \mathbf{p}^{(k)}\tag{CG-3}$$

其中 $\mathbf{x}^{(0)}=\mathbf{0}$ 为初始向量。

从 (CG-3) 可知，近似解 $\mathbf{x}^{(k+1)}$ 是由前一近似解 $\mathbf{x}^{(k)}$ 沿向量 $\mathbf{p}^{(k)}$ 的方向移动 $\alpha_k \mathbf{p}^{(k)}$ 得到的。若 $\alpha_k$ 由 (CG-3) 式确定，则有限步迭代(至多 n 步)即可得到方程组的精确解 $\mathbf{x}^\ast$。

注意到 $\mathbf{x}^{(k)}\in Span\{\mathbf{p}^{(0)},\cdots,\mathbf{x}^{(k-1)}\}$, 由 A 正交性 (CG-2)有 $(\mathbf{A}\mathbf{x}^{(k)}, \mathbf{p}^{(k)})=0$,所以

$$(\mathbf{b}, \mathbf{p}^{(k)})=((\mathbf{b} - \mathbf{A}\mathbf{x}^{(k)}),\mathbf{p}^{(k)})=(\mathbf{r}^{(k)},\mathbf{p}^{(k)})$$

这样 $\alpha_k$ 式可写成

$$\alpha_k = \frac{(\mathbf{r}^{(k)},\mathbf{p}^{(k)})}{(\mathbf{A}\mathbf{p}^{(k)},\mathbf{p}^{(k)})}$$

另一方面，设 $\mathbf{x}^{(0)}=\mathbf{0}$，且已有近似解 $\mathbf{x}^{(k)}\in Span\{\mathbf{p}^{(0)},\cdots,\mathbf{p}^{(k-1)}\}$,考虑极值问题：求 $\tilde{\alpha}_k$,使

$$\phi(\mathbf{x}^{(k)}+\tilde{\alpha}_k \mathbf{p}^{(k)})=\min_{\alpha}\phi(\mathbf{x}^{(k)}+\alpha\mathbf{p}^{(k)}),$$

其中二次函数 $\phi(\mathbf{x})=(\mathbf{A}(\mathbf{x}-\mathbf{x}^\ast),(\mathbf{x}-\mathbf{x}^\ast))$。由 $\phi(\mathbf{x})$ 的表达式有：

$$\phi(\mathbf{x}^{(k)}+\alpha\mathbf{p}^{(k)})=\phi(\mathbf{x}^{(k)})-2\alpha(\mathbf{p}^{(k)},\mathbf{r}^{(k)})+\alpha^2(\mathbf{p}^{(k)},\mathbf{A}\mathbf{p}^{(k)})$$

其中

$$\mathbf{r}^{(k)}=\mathbf{A}\mathbf{x}^\ast - \mathbf{A}\mathbf{x}^{(k)} = \mathbf{b}-\mathbf{A}\mathbf{x}^{(k)}$$

是近似解 $\mathbf{x}^{(k)}$ 的残量。由 $\frac{d\phi}{d\alpha}=0$ 得到

$$\tilde{\alpha}_k = \frac{(\mathbf{r}^{(k)},\mathbf{p}^{(k)})}{(\mathbf{A}\mathbf{p}^{(k)},\mathbf{p}^{(k)})}$$

此时 $\phi(\mathbf{x}^{(k)}+\alpha\mathbf{p}^{(k)})$ 达到极小。

由于 $\mathbf{x}^{(k)}(k=2,3,\cdots)$ 是由 $\mathbf{x}^{(1)}=\tilde{\alpha}_0\mathbf{x}^{(0)}$ 开始逐步求得的，所以

$$\mathbf{x}^{(k)}=\sum_{i=0}^{k-1}\tilde{\alpha}_i\mathbf{p}^{(i)}$$

比较上面的式子，知 $\tilde{\alpha}_k = \alpha_k$，这说明有限步迭代(CG-3)中的 $\alpha_k$ 正是使 $\phi(\mathbf{x}^{(k)}+\alpha\mathbf{p}^{(k)})$ 达到最小的参数。 

**构造共轭梯度**

下面将说明从任意初始向量 $\mathbf{x}^{(0)}$ 出发，可以逐次利用残量 $\{\mathbf{r}^{(k)}\}$ 来构造一组 A 共轭向量 $\{\mathbf{p}^{(k)}\}$ 和近似解 $\{\mathbf{x}^{(k)}\}$。因为 $\{\mathbf{r}^{(k)}\}$ 相应于 $\phi(\mathbf{x})$ 在 $\mathbf{x}=\mathbf{x}^{(k)}$ 处的梯度，所以 $\{\mathbf{p}^{(k)}\}$ 也称为共轭梯度。这样，利用 $\{\mathbf{p}^{(k)}\}$
来求解线性代数方程组的方法就称为**共轭梯度法**。

取 $\mathbf{p}^{(0)} = \mathbf{r}^{(0)}, \mathbf{p}^{(k)}(k=1,\cdots,n-1)$ 为与 $\mathbf{p}^{(0)},\cdots,\mathbf{p}^{(k-1)}$ 均 A 共轭的向量(这样的向量不是唯一的)。CG 法中取 $\mathbf{p}^{(k)}$ 为
$\mathbf{r}^{(k)}$ 与 $\mathbf{p}^{(k-1)}$ 的线性组合，即

$$\mathbf{p}^{(k)} = \mathbf{r}^{(k)} + \beta_{k-1}\mathbf{p}^{(k-1)}$$.

利用 $(\mathbf{p}^{(k)}, \mathbf{A}\mathbf{p}^{(k-1)}) = 0$ 得到

$$\beta_{k-1} = -\frac{(\mathbf{r}^{(k)}, \mathbf{A}\mathbf{p}^{(k-1)})}{(\mathbf{p}^{(k-1)}, \mathbf{A}\mathbf{p}^{(k-1)})}$$

这样确定的 $\mathbf{p}^{(k)}$ 与 $\mathbf{p}^{(k-1)}$ 是 A 共轭的。如此得到的向量序列 $\{\mathbf{p}^{(k)}\}$ 是一个 A共轭向量组。 

综上，可得如下算法：

1. 任取 $\mathbf{x}^{(0)}\in R^n$；


2. $\mathbf{r}^{(0)} = \mathbf{b}-\mathbf{A}\mathbf{x}^{(0)},\; \mathbf{p}^{(0)} = \mathbf{r}^{(0)}$;


3. 对于 $k=0,1,\cdots,$ 计算

$$\begin{split}
& \alpha_k = \frac{(\mathbf{r}^{(k)},\mathbf{p}^{(k)})}{(\mathbf{A}\mathbf{p}^{(k)},\mathbf{p}^{(k)})}\\
& \mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + \alpha_k \mathbf{p}^{(k)}\\
& \mathbf{r}^{(k+1)} = \mathbf{b} - \mathbf{A}(\mathbf{x}^{(k)}+\alpha_k \mathbf{p}^{(k)})= \mathbf{r}^{(k)}-\alpha_k \mathbf{A}\mathbf{p}^{(k)}\\
& \beta_{k} = -\frac{(\mathbf{r}^{(k+1)}, \mathbf{A}\mathbf{p}^{(k)})}{(\mathbf{p}^{(k)}, \mathbf{A}\mathbf{p}^{(k)})},\\
&\mathbf{p}^{(k+1)} = \mathbf{r}^{(k+1)} + \beta_k\mathbf{p}^{(k)}
\end{split}$$

由上面的式子可得

* $(\mathbf{r}^{(k+1)}, \mathbf{p}^{(k)}) = 0$;


* $(\mathbf{r}^{(k)}, \mathbf{p}^{(k)}) = (\mathbf{r}^{(k)}, \mathbf{r}^{(k)})$。

在计算过程中，若遇到 $\mathbf{r}^{(k)}=\mathbf{0}$ 或 $(\mathbf{p}^{(k)}, \mathbf{A}\mathbf{p}^{(k)})=0$，计算终止。此时：

* 如果 $\mathbf{r}^{(k)}=\mathbf{0}$ 则 $\mathbf{x}^{(k)}=\mathbf{x}^\ast$;


* 如果 $(\mathbf{p}^{(k)}, \mathbf{A}\mathbf{p}^{(k)})=0$，则因 $\mathbf{A}$ 正定，有 $\mathbf{p}^{(k)}= \mathbf{0}$。再由 $(\mathbf{r}^{(k)}, \mathbf{r}^{(k)})=(\mathbf{r}^{(k)}, \mathbf{p}^{(k)}) =\mathbf{0}$,所以亦有 $\mathbf{r}^{(k)}=\mathbf{0}$。

# BFGS(拟牛顿法)

Broyden–Fletcher–Goldfarb–Shanno（BFGS）算法具有牛顿法的一些优点，但没有牛顿法的计算负担。

我们回顾牛顿法，由下式给出
$$\mathbf{\theta}^\ast = \mathbf{\theta}_0-\mathbf{H}^{-1}\nabla_{\mathbf{\theta}}J(\mathbf{\theta}_0)$$

其中 $\mathbf{H}$ 是 J 相对于 $\mathbf{\theta}$ 的 Hessian 矩阵在 $\mathbf{\theta}_0$ 处的估计。

运用牛顿法的主要计算难点在于计算 Hessian 逆 $\mathbf{H}^{-1}$。拟牛顿法所采用的方法(BFGS是其中最突出的)是使用矩阵 $\mathbf{M}_t$ 近似逆，迭代更新以更好近似 $\mathbf{H}^{-1}$。

当 Hessian 逆近似 $\mathbf{M}_t$ 更新时，下降方向 $\mathbf{M}_t\mathbf{g}_t$。参数更新为

$$\mathbf{\theta}_{t+1} = \mathbf{\theta}_{t}+\epsilon^\ast\mathbf{M}_t\mathbf{g}_t$$
$\epsilon^\ast$ 是搜索步长。

该方法和共轭梯度不同的是：

* 该方法的成功并不严重依赖于线搜索寻找该方向上和真正极小值很近的一点；


* 相比于 共轭梯度，BFGS 的优点是其花费较少的时间改进每个线搜索；


* BFGS 算法必须存储 Hessian 逆矩阵 $\mathbf{M}$,需要 $O(n^2)$ 的存储空间，使 BFGS 不适用于大多数具有百万级参数的现代深度学习模型。

***
**存储受限的 BFGS(或L-BFGS)** 通过避免存储完整的 Hessian 逆矩阵 $\mathbf{M}$， BFGS 算法的存储代价可以显著降低。L-BFGS 算法使用和 BFGS 算法相同的方法计算 $\mathbf{M}$ 的近似，但是其实假设 $\mathbf{M}^{(t-1)}$ 是单位矩阵，而不是一步一步存储近似。

# [Lagrange 对偶与KKT条件](https://wizardforcel.gitbooks.io/dm-algo-top10/content/svm-2.html)

对于 向量 $\mathbf{x}\in R^n$，考虑一般形式的不等式约束问题：

$$\left\{\begin{aligned}
\min_{\mathbf{x}} f(\mathbf{x}) &\\
s.t. h_i(\mathbf{x}) = 0 &\quad i = 1,2,...,k \\
g_i(\mathbf{x}) \leq  0 &\quad i = 1,2,...,l
\end{aligned}\right. \tag{Primal Constraints}$$

对应的 Lagrange 公式为：

$$L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = f(\mathbf{x}) + \sum_{i=1}^{k}\alpha_i g_i(\mathbf{x}) + \sum_{i=1}^{l}\beta_i h_i(\mathbf{x}) \tag{Larange Model}$$

其中 $\mathbf{\alpha}, \mathbf{\beta}$ 都是 Lagrange 算子。

我们要求解的就是满足 **primal constraints (简称PC)** 的 $\min_{\mathbf{x}}f(\mathbf{x})$。

对于 $L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta})$ 我们适当地调整 $\mathbf{\alpha}, \mathbf{\beta}$ 的值，总会有：

$$ \max_{\beta,\alpha \geq 0} L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta})=
\left \{
\begin{aligned}
f(\mathbf{x}) & \quad (\mathbf{\alpha}, \mathbf{\beta}) \; satisfy \; (PC)\\
+\infty & \quad else 
\end{aligned}\right.$$

如果记 $\theta_{PC}(\mathbf{x}) = \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0} L(x, \mathbf{\alpha}, \mathbf{\beta})$，则求解满足 PC 的 $\min_{\mathbf{x}}f(\mathbf{x})$的问题等价于

$$\begin{split}
d^\ast = \min_{\mathbf{x}}f(\mathbf{x}) =& \min_{\mathbf{x}} \theta_{PC}(\mathbf{x})\\
=& \min_{\mathbf{x}} \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0} L(\mathbf{x}, \alpha, \beta)
\end{split}$$

一般有 $\max\min f \leq \min\max f$ (从 $f \leq \max f$开始推导) ,所以有：

$$ d^\ast = \min_{\mathbf{x}}f(\mathbf{x}) =\min_{\mathbf{x}} \max_{\mathbf{\beta}, \mathbf{\alpha} \geq 0} L(\mathbf{x}, \alpha, \beta)
\geq \max_{\mathbf{\beta}, \mathbf{\alpha}\geq 0}\min_{\mathbf{x}} L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = p^\ast$$

到此为止，求解不等式左侧满足约束的极小值问题转换成不等式右侧求极大值的问题。求极大值是凸规划里的常见问题，对偶问题的转换都是往利于求解的方向。

**KKT条件：**

求解满足 （PC）的极小值问题，对应的 Lagrange 公式存在极小值的必要条件是满足KKT条件：

$$\left\{\begin{aligned}
\nabla_{\mathbf{x}}  L(\mathbf{x}, \mathbf{\alpha}, \mathbf{\beta}) = \mathbf{0}\\
h_i(\mathbf{x}) = 0 \\
g_i(\mathbf{x}) \leq  0  \\
\alpha_i \geq 0 \\
\alpha_i g_i = 0
\end{aligned}\right. \tag{KKT}$$


# Newton-Raphson切线法

逐点线性化方法是构造迭代格式的重要方法。从几何上看，解方程 $f(x)=0$ 可以看做是求曲线 $y=f(x)$ 与 x 轴的交点。

**以直代曲**

在任何一点x附近（特别当 x 靠近 $x^\ast$ 的时候），曲线可以近似看做直线，从而可以**用线性方程近似代替曲线方程，用直线方程的根去近似曲线方程的根。这种以直线代替曲线，或以线性方程的根代替曲线方程的根的方法，就是求解各种非线性方程的一个最基本的原理，即所谓的“线性化”原理。**

下面我们介绍切线法。

给定方程

$$f(x)=0\tag{QX}$$

用迭代法求解（QX）归结为构造 $\varphi(x)$,使相应迭代序列

$$x_{k+1}=\varphi(x_k)$$

收敛于（QX）的解 $x^\ast$。

设 f(x) 在 $x^\ast$ 附近**二次连续可微(这个条件是为了保证收敛性)**，选 $x^\ast$ 的一个近似 $x_0$，在 $x_0$ 点将 f(x) 展成 Taylor 级数

$$f(x) = f(x_0) + f^{'}(x_0)(x-x_0) + \frac{1}{2!}f^{''}(\xi_0)(x-x_0)^2$$

其中 $\xi_0$ 是满足 $\vert \xi_0-x_0\vert < \vert x-x_0\vert$ 的一点。

由于 $x_0$ 非常接近 $x^\ast$，所以可以用线性方程来近似求解曲线方程 (QX)：

$$f(x_0) + f^{'}(x_0)(x-x_0)=0$$


注：实际操作中，我们无法知道其是否非常接近真实解，所以起始点的选取要通过其他方法来判断，比如不要选在某个极值点，对应的一阶导为0，无法迭代下去。

其解 

$$x_1 = x_0 - \frac{f(x_0)}{f^{'}(x_0)}$$

可以看成对 $x^\ast$ 新的近似。

如此继续下去，就可以得到切线法的迭代序列：

$$x_{k+1} =x_k - (f^{'}(x_k))^{-1}f(x_k)$$

<img src='figure/Newton-Raphson0.png' width='400' height='300' align='left'>

下面通过对比二分法和切线法来求解正整数 M 的平方根，即曲线方程为 $x^2-M=0$,要求解的精度也就是误差范围在 $10^{-4}$。

In [23]:
# 二分法

def split_into_two_sqrt(left,right, target, error):
    m = (left+right)/2.0
    err = m**2 - target
    count = 0
    while abs(err)> error:
        if err < 0:
            left = m
        else:
            right = m
        m = (left+right)/2.0
        err = m**2 - target
        count +=1
#     print('the result is {};run count is {}'.format(m,count))
    return m,count           

Newton-Raphson切线法对应的迭代公式为：

$x_{k+1} =x_k - \frac{x_k^2\;-M}{2x_k}$

In [25]:
# Newton-Raphson切线法

def newton_raphson(begin,target,error):
    err = begin**2 - target
    count = 0
    while abs(err)> error:
        begin -= (begin**2 - target)/(2*begin)
        err = begin**2 - target
        count +=1
#     print('the result is {};run count is {}'.format(begin,count))
    return begin,count 

In [43]:
result_list = []
target_list = [2**i for i in range(1,10)]
for i in target_list:
    begin,count_1 = newton_raphson(begin=i,target=i,error=10**(-8))
    x,count_2=split_into_two_sqrt(left=0,right=i-0.5, target=i,error=10**(-8))
    result_list.append([i,begin, count_1, count_2])

In [45]:
import pandas as pd
pd.DataFrame(result_list, columns=['target','sqrt_result', 'newton', 'split_two'])

,target,sqrt_result,newton,split_two
0,2,1.414214,4,27
1,4,2.000000,5,27
2,8,2.828427,5,26
3,16,4.000000,6,31
4,32,5.656854,7,33
5,64,8.000000,7,30
6,128,11.313708,8,36
7,256,16.000000,8,30
8,512,22.627417,9,39
